In [1]:
import pandas as pd
import numpy as np
import pathlib
import datetime
from datetime import datetime, timedelta, date
from rich import print
from collections import namedtuple
from dataclasses import dataclass, asdict
import operator
import tomllib
import tomlkit as tml
import json
import xarray as xr
from csvb.ingest import apply_rule, sel_factory, ApplyOp, SelectOp, AOP, SOP, Rule, to_transactions, strip_currency
#from decimal import Decimal, getcontext

#getcontext().prec = 2
pd.options.mode.copy_on_write = True

In [2]:
def accounts(trans):
    return set(trans["From"]).union(set(trans["To"]))

def read_and_apply(df, rulepth, debug=False):
    with open(rulepth) as f:
        for rule in tml.load(f)["rules"]:
            if debug:
                print(rule)
            apply_rule(Rule(**rule), df)
    return df
    

In [3]:
home = pathlib.Path.home()
cfgpth = home / ".config" / "csvb" / "config.toml"
with open(cfgpth, 'rb') as f:
    cfg = tomllib.load(f)

In [4]:
try:
    del apl
except NameError:
    pass

apl = pd.read_csv(home / pathlib.Path(cfg["paths"]["bank"]) / "Jan-2024_Apple_CC.csv",
                    header=0, parse_dates=["Transaction Date", "Clearing Date"])

apl = apl.rename(columns={"Amount (USD)": "Amount"})
apl["Amount"] = -1 * apl["Amount"]
apl["From"] = ""

apl = read_and_apply(apl, home / cfg["paths"]["rules"] / "aplrules.toml")
#apl

In [5]:
try:
    del fst
except NameError:
    pass

fst = pd.read_csv(home / pathlib.Path(cfg["paths"]["bank"]) / "Jan-2024_1st.csv",
                    header=3, parse_dates=["Date"])

fst["Amount"] = np.nansum([fst["Amount Credit"],fst["Amount Debit"]], axis=0)
fst["From"] = ""

fst = read_and_apply(fst, home / cfg["paths"]["rules"] / "fstrules.toml")
#fst

In [6]:
try:
    del ally
except NameError:
    pass

ally = pd.read_csv(home / pathlib.Path(cfg["paths"]["bank"]) / "Jan-2024_Ally.csv",
                    header=0, parse_dates=["Date"])

ally["From"] = ""
ally = ally.rename(columns=dict(zip(ally.columns, [c.strip() for c in ally.columns])))

ally = read_and_apply(ally, home / cfg["paths"]["rules"] / "allyrules.toml")
#ally

In [7]:
try:
    del ndfcu
except NameError:
    pass

ndfcu = pd.read_csv(home / pathlib.Path(cfg["paths"]["bank"]) / "Jan-2024_NDFCU_CC.csv",
                    header=0, names=["Date", "Note", "Description", "Amount", "Balance"],
                    parse_dates=["Date"])

ndfcu["From"] = ""
ndfcu["Amount"] = ndfcu["Amount"].apply(strip_currency).astype(float)
#ndfcu = ally.rename(columns=dict(zip(ally.columns, [c.strip() for c in ally.columns])))
ndfcu = read_and_apply(ndfcu, home / cfg["paths"]["rules"] / "ndfcurules.toml")
#ndfcu

In [8]:
bank_tables = {"Assets:1st Source:Checking": fst,
               "Liabilities:Apple:Credit Card": apl.rename(columns={"Transaction Date": "Date"}),
              "Assets:Ally:Savings": ally,
              "Liabilities:NDFCU:Credit Card": ndfcu}


trans = to_transactions(bank_tables)
trans

,Date,Description,Amount,From,To
0,2023-12-18,Interest Paid,32.85,Income:Interest,Assets:Ally:Savings
1,2023-12-31,Netflix.com . netflix.com . CA 0000,-15.49,Expenses:Subscriptions,Liabilities:NDFCU:Credit Card
2,2024-01-01,BOOKSHOP.ORG 235 WITHERS STREETUNIT 2 BROOKLYN...,-29.09,Expenses:Study,Liabilities:Apple:Credit Card
3,2024-01-01,PATREON* MEMBERSHIP 600 TOWNSEND 5TH FLOOR INT...,-27.19,Expenses:Entertainment,Liabilities:Apple:Credit Card
4,2024-01-01,AB* ABEBOOKS.CO K7JNI5410 TERRY AVENUE NORTH S...,-48.83,Expenses:Study,Liabilities:Apple:Credit Card
...,...,...,...,...,...
80,2024-01-31,MONTHLY INSTALLMENTS (21 OF 24),-54.12,Liabilities:Apple:Installments,Liabilities:Apple:Credit Card
81,2024-01-31,ACH DEPOSIT INTERNET TRANSFER FROM ACCOUNT END...,2315.13,Assets:1st Source:Checking,Liabilities:Apple:Credit Card
82,2024-01-31,APPLE CASH PAYMENT,40.45,Income:Apple Cash,Liabilities:Apple:Credit Card
83,2024-01-31,crossfit south b crossfit s,-80.00,Expenses:Groceries,Assets:1st Source:Checking


In [9]:
try:
    del bal_decl
except NameError:
    pass

bal_decl = pd.read_csv(home / pathlib.Path(cfg["paths"]["bank"]) / "balances.csv",
                  header=0,
                  parse_dates=["Date"])

#bal_decl["From"] = ""
#bal["Amount"] = bal["Amount"].apply(strip_currency).astype(float)
#ndfcu = ally.rename(columns=dict(zip(ally.columns, [c.strip() for c in ally.columns])))
#bal_decl = bal_decl.sort_values("Date")
#bal_decl
#jan_bal = bal_decl.loc[(bal_decl["Date"] >= pd.Timestamp(2023, 12, 31)) &
#                       (bal_decl["Date"] <= pd.Timestamp(2024, 1, 1))
#trans = pd.concat([trans, bal_decl])
bal_decl

,Date,Account,Statement Balance
0,2022-12-31,Liabilities:NDFCU:Credit Card,-76.69
1,2023-11-30,Liabilities:NDFCU:Credit Card,-2085.08
2,2023-10-31,Liabilities:NDFCU:Credit Card,-2492.89
3,2023-09-30,Liabilities:NDFCU:Credit Card,-1581.47
4,2023-08-31,Liabilities:NDFCU:Credit Card,-69.79
5,2023-07-31,Liabilities:NDFCU:Credit Card,-3172.26
6,2023-06-30,Liabilities:NDFCU:Credit Card,-1409.14
7,2023-05-31,Liabilities:NDFCU:Credit Card,-351.48
8,2023-04-30,Liabilities:NDFCU:Credit Card,-3924.01
9,2023-03-31,Liabilities:NDFCU:Credit Card,-2519.52


In [10]:
def create_ledgers(trans):
    """ Create a ledger for each account of all from / to transactions and create balance / time column.
    'Transaction Pair' is a terrible name, but it reminds that this is 1 half of a known transation for use
    as a ledger of an account and running balance calculation.
    All values appended from the "From" side of a transaction are reversed here, because a positive value
    "From" this account subtracts from it's balance.
    
    """
    # TODO: Add an option of a start of / end of period dates. Automatically calculate balance at
    # beginning of period.
    # TODO: Add processing of balance assertions to allow checks and reconciliation.
    
    fl = trans.groupby("From")
    tl = trans.groupby("To")
    ledgers = {}
    for acct in accounts(trans):
        # From accounts, note the negative applied to Amount.
        try:
            df = pd.DataFrame(trans.loc[fl.groups[acct], ["Date", "Description", "To"]]).rename(columns={"To": "Transaction Pair"})
            df["Incoming Amount"] = -1 * trans.loc[fl.groups[acct], "Amount"]
            ledgers[acct] = df
        # If account is not in From group, skip.
        except KeyError:
            pass

        # To accounts.

        try:
            df = pd.DataFrame(trans.loc[tl.groups[acct], ["Date", "Description", "From"]]).rename(columns={"From": "Transaction Pair"})
            df["Incoming Amount"] = trans.loc[tl.groups[acct], "Amount"]
            try:
                ledgers[acct] = pd.concat([ledgers[acct], df])
            # If account was not also in From accounts list, don't concat.
            except KeyError:
                ledgers[acct] = df
        # If account is not in list To accounts, skip.
        except KeyError:
            pass
            
        # Sort and apply cumulative sum.
        ledgers[acct] = ledgers[acct].sort_values("Date")
        ledgers[acct]["Balance"] = ledgers[acct]["Incoming Amount"].cumsum()

    return ledgers

def trans_to_ledger(trans, acct, bal_decl = None, clean=False, leq=True):
    """ Craete a single ledger. This is a step to using ledgers as a view on the transactions rather than
    a precalcualted object.
    
    """   
    fl = trans.groupby("From")
    tl = trans.groupby("To")
    #ledgers = {}
    #for acct in accounts(trans):
    # From accounts, note the negative applied to Amount.
    try:
        df = pd.DataFrame(trans.loc[fl.groups[acct], ["Date", "Description", "To"]]).rename(columns={"To": "Transaction Pair"})
        df["Incoming Amount"] = -1 * trans.loc[fl.groups[acct], "Amount"]
        ledger = df
        # If account is not in From group, skip.
    except KeyError:
        pass

    # To accounts.

    try:
        df = pd.DataFrame(trans.loc[tl.groups[acct], ["Date", "Description", "From"]]).rename(columns={"From": "Transaction Pair"})
        df["Incoming Amount"] = trans.loc[tl.groups[acct], "Amount"]
        try:
            ledger = pd.concat([ledger, df])
        # If account was not also in From accounts list, don't concat.
        except (NameError, UnboundLocalError):
            ledger = df
    # If account is not in list To accounts, skip.
    except KeyError:
        pass
    if clean:
        ledger = clean_ledger(ledger)        
    # Sort and apply cumulative sum.
    
    if bal_decl is not None:
        ledger = append_init_row(ledger, acct, bal_decl, leq=leq)
    ledger = ledger.sort_values("Date")
    ledger["Balance"] = ledger["Incoming Amount"].cumsum()

    return ledger

def clean_ledger(ledger):
    """Remove transactions occurring before the last transaction with all accounts assigned.
    Intended to catch incomplete data and return a useful ledger set on a per account basis.
    """
    #clean = {}
    #for acct in ledgers:
    try:
        clean = ledger.loc[[ledger["From"] == ""]:,:]
    except KeyError:
        clean = ledger
        
    return clean

def init_balance(bal_decl, acct, first_date, leq=False):
    #first_date = min(ledger["Date"])
    if leq:
        acct_balances = bal_decl.loc[(bal_decl["Account"] == acct) & (bal_decl["Date"] <= first_date)]
    else:
        acct_balances = bal_decl.loc[(bal_decl["Account"] == acct) & (bal_decl["Date"] < first_date)]
    
    # Catch indexerror if no balance found.
    try:
        return acct_balances.sort_values("Date").iloc[-1]
    except IndexError:
        return None

def append_init_row(ledger, acct, bal_decl, leq=False):
    #print(acct)
    #print(ledger)
    # Find proper initial balance.
    init = init_balance(bal_decl, acct, min(ledger["Date"]), leq=leq)

    # Skip if no appropriate balance was found.
    if init is not None:
        
        # Create df of initial balance row.
        decl_to_ledg_col = {"Statement Balance": "Incoming Amount", "Account": "Transaction Pair"}
        init_df = pd.DataFrame(init).T.rename(columns=decl_to_ledg_col)
        init_df["Description"] = "Initial Balance"

        # Add to first row, make sure we are sorted before recalculating balance column.
        ledger = pd.concat([init_df, ledger]).sort_values("Date")
        ledger["Balance"] = ledger["Incoming Amount"].cumsum()
    return ledger  
        

#ledgers = create_ledgers(trans)
ledgers = {}
for acct in accounts(trans):
    ledgers[acct] = trans_to_ledger(trans, acct, bal_decl=bal_decl, clean=True)

In [13]:
bals = []
firstds = []
lastds = []
initbals = []

for acct in ledgers:
    bals.append(ledgers[acct]["Incoming Amount"].sum())
    firstds.append(ledgers[acct]["Date"].iloc[0])
    lastds.append(ledgers[acct]["Date"].iloc[-1])
    initial = ledgers[acct].loc[ledgers[acct]["Description"] == "Initial Balance"]
    if len(initial) != 0:
        initbals.append(initial["Balance"].iloc[0])
    else:
        initbals.append(0)
    #initbals.append(ledgers[acct]["Balance"].iloc[0])
    
acct_bals = pd.DataFrame({"Period Start": firstds,
                          "Initial Balance": initbals,
                          "Period End": lastds,
                          "Ending Balance": bals},
                         index = ledgers.keys())


In [14]:
acct_bals["Difference"] = acct_bals["Ending Balance"] - acct_bals["Initial Balance"]
acct_bals

,Period Start,Initial Balance,Period End,Ending Balance,Difference
Expenses:Subscriptions,2023-12-31,0.00,2024-01-30,55.97,55.97
Income:Notre Dame:Regular,2024-01-31,0.00,2024-01-31,-5335.70,-5335.70
Expenses:Entertainment,2024-01-01,0.00,2024-01-19,56.18,56.18
Expenses:Entertainment:Self,2024-01-01,0.00,2024-01-18,142.54,142.54
Expenses:Auto:Gas,2024-01-26,0.00,2024-01-26,8.43,8.43
Liabilities:Apple:Credit Card,2023-12-31,-284.77,2024-01-31,-62.41,222.36
Expenses:Groceries,2024-01-01,0.00,2024-01-31,1031.34,1031.34
Expenses:Home:Mortgage,2024-01-02,0.00,2024-01-02,746.58,746.58
Expenses:Gifts,2024-01-03,0.00,2024-01-03,125.00,125.00
Expenses:Gear,2024-01-24,0.00,2024-01-24,101.65,101.65


In [12]:
ledgers["Assets:1st Source:Checking"].loc[ledgers["Assets:1st Source:Checking"]["Description"] == "Initial Balance"]
#ledgers["Assets:1st Source:Checking"]["Description"]

,Date,Transaction Pair,Incoming Amount,Description,Balance
25,2023-12-31 00:00:00,Assets:1st Source:Checking,2320.59,Initial Balance,2320.59


In [35]:
ledgers["Assets:1st Source:Checking"]

,Date,Transaction Pair,Incoming Amount,Description,Balance
25,2023-12-31 00:00:00,Assets:1st Source:Checking,2320.59,Inital Balance,2320.59
12,2024-01-02 00:00:00,Expenses:Home:Mortgage,-746.58,DOVENMUEHLE MTG MORTG PYMT,1574.01
14,2024-01-02 00:00:00,Expenses:Groceries,-40.0,crossfit south b crossfit s,1534.01
22,2024-01-03 00:00:00,Expenses:Gifts,-125.0,VENMO PAYMENT,1409.01
23,2024-01-04 00:00:00,Expenses:Groceries,-3.0,crossfit south b crossfit s,1406.01
27,2024-01-09 00:00:00,Expenses:Utilities,-110.57,VERIZON WIRELESS PAYMENTS,1295.44
28,2024-01-09 00:00:00,Transfer:Ally:Savings,-600.0,ALLY BANK $TRANSFER,695.44
43,2024-01-16 00:00:00,Expenses:Groceries,-3.0,crossfit south b crossfit s,692.44
44,2024-01-16 00:00:00,Transfer:Ally:Savings,1000.0,KEITH W DAVIS P2P,1692.44
45,2024-01-17 00:00:00,Expenses:Utilities,-84.27,AEP INDIANA MICH BILL PAY,1608.17


In [27]:
ledgers.keys()

dict_keys(['Assets:1st Source:Checking', 'Liabilities:NDFCU:Credit Card', 'Expenses:Home:Mortgage', 'Expenses:Clothing', 'Liabilities:Apple:Credit Card', 'Expenses:Study', 'Expenses:Home:Cleaning', 'Expenses:Utilities', 'Expenses:Medical:Therapy', 'Expenses:Auto:Tolls', 'Expenses:Subscriptions', 'Expenses:Entertainment', 'Liabilities:Apple:Installments', 'Transfer:Ally:Savings', 'Expenses:Home', 'Expenses:Gifts', 'Expenses:Pet', 'Expenses:Auto:Fuel', 'Expenses:Subscription', 'Expenses:Groceries', 'Expenses:Medical:Body', 'Income:Apple Cash', 'Expenses:Entertainment:Reading', 'Expenses:Entertainment:Self', 'Expenses:Gear', 'Assets:Ally:Savings', 'Income:Notre Dame:Regular', 'Expenses:Auto:Gas', 'Expenses:Restaurants', 'Income:Interest'])

10      35.00
14      75.00
16     213.98
23     216.98
39     304.65
43     307.65
63     532.65
66     741.34
70     951.34
83    1031.34
Name: Balance, dtype: float64